# 📊 Interpretation der Anonymisierungs-ErgebnisseDieses Notebook enthält die detaillierte Interpretation der Ergebnisse aus dem vorherigen Anonymisierungsprozess (k-Anonymität, l-Diversität, t-Closeness) im Kontext eines Retail-Datensatzes. Der Prozess basierte auf den Schutzprinzipien der k-Anonymität, l-Diversität und t-Closeness, mit dem Ziel, die Daten für analytische Zwecke nutzbar zu machen, während das Risiko der Re-Identifikation gemäß DSGVO minimiert wird.

## 1. Datenvorbereitung und Generalisierung (Der Schutzschild)---Der Prozess begann mit der **Filterung des Originaldatensatzes**, bei dem alle direkten Identifikatoren (wie Transaktions-ID, Name, E-Mail etc.) entfernt wurden. Es verblieben lediglich die **Quasi-Identifikatoren (QI)** (`Zipcode`, `Age`, `Country`) und das **Sensible Attribut (SA)** (`Income`).Der kritische erste Schritt zur Anonymisierung war die **Generalisierung der Quasi-Identifikatoren**:* **Postleitzahl (`Zipcode`)**: Die exakte PLZ wurde auf die ersten **zwei Ziffern** reduziert, um die Äquivalenzklassen zu vergrößern.* **Alter (`Age`)**: Das genaue Alter wurde in breite **Intervalle** (`<=30`, `31-50`, `51+`) gruppiert, um die Anonymitätsvoraussetzungen zu schaffen.

## 2. Überprüfung der Anonymisierungs-Kriterien---Der Schutzgrad wurde schrittweise überprüft.### k-Anonymität ($k=2$)Das Kriterium ($k=2$) besagt, dass jede Kombination der Quasi-Identifikatoren mindestens 2 Datensätze umfassen muss.* **Ergebnis**: 301.837 von 302.010 Datensätzen verblieben (**99,94 %** Erhaltung).* **Interpretation**: Die Generalisierung war sehr effektiv; nur ein minimaler Teil (0,06 %) musste entfernt werden. Das Re-Identifikationsrisiko basierend auf den QIs ist stark reduziert.### l-Diversität ($l=2$)Die $l$-Diversität ($l=2$) stellt sicher, dass jede Äquivalenzklasse **mindestens 2 verschiedene Werte** des Sensiblen Attributs (`Income`) enthält, was Homogenitätsangriffe verhindert.* **Ergebnis**: 301.837 von 301.837 (k-anonymen) Datensätzen verblieben (**100,00 %** Erhaltung).* **Interpretation**: Die Bandbreite der Einkommensklassen war ausreichend, um diesen Schutzstandard ohne weiteren Datenverlust zu gewährleisten.### t-Closeness ($t=0.3$)Die $t$-Closeness stellt sicher, dass die **Verteilung** des Sensiblen Attributs (`Income`) innerhalb jeder Äquivalenzklasse nicht zu stark (max. Euklidische Distanz $t=0.3$) von der Gesamtverteilung abweicht. Dies schützt vor Attribut-Inferenzangriffen.* **Ergebnis**: 301.837 von 301.837 (l-diversen) Datensätzen verblieben (**100,00 %** Erhaltung).* **Interpretation**: Die Verteilung der Einkommen in den Gruppen weicht in keiner Weise signifikant von der Gesamtverteilung ab, was die Daten für Machine Learning oder Business Intelligence **besonders wertvoll** macht.

## 3. Analyse des Trade-offs (Sicherheitsgrad vs. Datenverlust)---Die folgende Code-Zelle visualisiert, wie sich eine Erhöhung des Sicherheitsgrads ($k$) auf die verbleibende Datenmenge auswirkt. Der zugrunde liegende Datensatz ist die generalisierte Datei (`retail_data_filtered.csv`).| $k$-Wert | Verbliebene Datensätze | Erhaltungsrate || :--- | :--- | :--- || $k=2$ | 301.837 | 99.94 % || $k=3$ | 301.699 | 99.89 % || $k=5$ | 301.621 | 99.87 % || $k=10$ | 301.554 | 99.85 % || $k=20$ | 301.422 | 99.81 % |Die Grafik zeigt, dass der Datenverlust **sehr gering** ist, selbst bei einer drastischen Erhöhung auf $k=20$.

In [ ]:
import pandas as pdimport matplotlib.pyplot as pltimport osdef get_k_anonymized_count(df_input, quasi_identifiers, k_val):    # Gruppengrößen berechnen    group_sizes = df_input.groupby(quasi_identifiers, observed=True).size().reset_index(name='Count')    # Filterung und Anzahl der verbleibenden Datensätze    df_k_anonym = df_input.merge(group_sizes[group_sizes['Count'] >= k_val],                                 on=quasi_identifiers)    return len(df_k_anonym)# Pfad zur generalisierten Datei (Voraussetzung: vorheriger Code wurde ausgeführt)CSV_PATH = "./data/retail_data_filtered.csv"# Daten laden und Generalisierung erneut anwendenif not os.path.exists(CSV_PATH):    print(f"Fehler: Die Datei '{CSV_PATH}' wurde nicht gefunden. Bitte stellen Sie sicher, dass die vorherigen Code-Schritte ausgeführt wurden.")    df_generalized = Noneelse:    df_generalized = pd.read_csv(CSV_PATH)    if df_generalized is not None:    # Generalisierungsschritte replizieren (muss exakt zur Analyse passen)    df_generalized['Zipcode'] = df_generalized['Zipcode'].astype(str).str[:2]    df_generalized['Age'] = pd.cut(df_generalized['Age'], bins=[0, 30, 50, 100],                       labels=['<=30', '31-50', '51+'], include_lowest=True)        quasi_identifiers = ['Zipcode', 'Age', 'Country']    k_values = [2, 3, 5, 10, 20]    remaining_counts = []    total_records = len(df_generalized)    print(f"Anzahl Datensätze Original: {total_records}")        for k_val in k_values:        count = get_k_anonymized_count(df_generalized, quasi_identifiers, k_val)        remaining_counts.append(count)        print(f"Für k={k_val}: {count} Datensätze ({count/total_records:.2%} erhalten)")        # Visualisierung des Trade-offs    plt.figure(figsize=(10, 6))    plt.plot(k_values, remaining_counts, marker='o', linestyle='-', color='red')    plt.title('Trade-off: k-Anonymität vs. verbleibende Datensätze')    plt.xlabel('Anonymitätsgrad k')    plt.ylabel('Anzahl der verbleibenden Datensätze')    plt.grid(True, linestyle='--', alpha=0.6)    plt.show()

## Fazit---Der durchgeführte Anonymisierungsprozess war ein voller Erfolg:1.  **Hoher Datenschutz**: Alle drei strengen Anonymisierungskriterien ($k=2$, $l=2$, $t=0.3$) wurden erfüllt.2.  **Minimierter Datenverlust**: Es konnten beeindruckende **99,94 %** der Originaldatensätze erfolgreich anonymisiert werden, was die **statistische Aussagekraft** der Daten nahezu vollständig erhält.3.  **Effizienz der Generalisierung**: Die einfache, aber effektive Generalisierung von PLZ und Alter war der Schlüssel, um diesen hohen Schutzgrad mit minimalem Datenverlust zu erreichen.Der resultierende Datensatz ist somit optimal für eine DSGVO-konforme Weiterverarbeitung geeignet, da er ein ausgezeichnetes Gleichgewicht zwischen **Datennutzen** und **Datenschutz** bietet.